In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
from matplotlib.colors import ListedColormap

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

from pathlib import Path

## Introduction
In this lab we will use both supervised (with labels) and unsupervised techniques to attempt to create a predictor for heart failure. The original data set can be viewed at https://www.kaggle.com/andrewmvd/heart-failure-clinical-data

### Prepare data

In [ ]:
#DATA_PATH = Path(r"data")
#data_file = DATA_PATH / "heart_failure_clinical_records_dataset.csv"

# Run the following if running with Colab
data_file = "https://github.com/fordanic/cmiv-ai-course/raw/master/notebooks/data/heart_failure_clinical_records_dataset.csv"

In [ ]:
df = pd.read_csv(data_file)
df.head()

We use 'DEATH_EVENT' as the categorical target, so it should be removed from the feature set we can use for prediction. We also remove the 'time' feature, since it seems unfairly correlated with the target.

In [ ]:
X = df.drop(['DEATH_EVENT', 'time'], axis=1)
y_cat = df['DEATH_EVENT']
y_reg = df['time']

### (Option)
Optionally we could normalize all numerical features to have zero mean and variance one. This does not seem to make a huge difference.

In [ ]:
#num_cols = ['age', 'creatinine_phosphokinase', 'ejection_fraction', 'platelets', 'serum_creatinine', 'serum_sodium']
#X[num_cols] -= X[num_cols].mean()
#X[num_cols] /= X[num_cols].std()
#X.head()

In [ ]:
len(np.where(y_cat == 1)[0]), len(np.where(y_cat == 0)[0])

### Helpers

We define a few helpers to quickly be able to iterate experiments.

In [ ]:
colors = ['red', 'green']
def plot_pca(X):
    pca = PCA(n_components=2)
    x_pca = pca.fit_transform(X)
    plt.scatter(x_pca[:, 0],x_pca[:, 1], c=y_cat, cmap =ListedColormap(colors))

For each experiment we accept a list of features to use. We run a PCA (dimensionality reduction to 2D) on the data and plot the result to visualize the main correlation among the selected features.

In [ ]:
def run_experiment(features=None):
    print(f"Using features {features}")
    if features is not None:
        X_f = X[features]
    else:
        X_f = X
    
    plot_pca(X_f)
    
    X_train, X_test, y_train, y_test = train_test_split(X_f, y_cat, random_state=42, test_size=.30)
    lor = LogisticRegression(random_state=0, class_weight="balanced")
    lor.fit(X_train, y_train)

    print(f'Score features: {round(lor.score(X_test, y_test),4)}')
    cm =confusion_matrix(y_test, lor.predict(X_test))
    ConfusionMatrixDisplay(cm, display_labels=["Healthy", "Heart failure"]).plot()

### Experiment

Try to find good combinations of features to predict heart failure. What are your conclusions?

In [ ]:
run_experiment(['age', 'creatinine_phosphokinase'])